In [1]:
import numpy as np
import torch
import os

In [2]:
def read2Dspikes(filename):
    '''
    Reads two dimensional binary spike file and returns a TD event.
    It is the same format used in neuromorphic datasets NMNIST & NCALTECH101.
    
    The binary file is encoded as follows:
        * Each spike event is represented by a 40 bit number.
        * First 8 bits (bits 39-32) represent the xID of the neuron.
        * Next 8 bits (bits 31-24) represent the yID of the neuron.
        * Bit 23 represents the sign of spike event: 0=>OFF event, 1=>ON event.
        * The last 23 bits (bits 22-0) represent the spike event timestamp in microseconds.
    Arguments:
        * ``filename`` (``string``): path to the binary file.
    '''
    with open(filename, 'rb') as inputFile:
        inputByteArray = inputFile.read()
    inputAsInt = np.asarray([x for x in inputByteArray])
    xEvent =   inputAsInt[0::5]
    yEvent =   inputAsInt[1::5]
    pEvent =   inputAsInt[2::5] >> 7
    tEvent =( (inputAsInt[2::5] << 16) | (inputAsInt[3::5] << 8) | (inputAsInt[4::5]) ) & 0x7FFFFF
    return event(xEvent, yEvent, pEvent, tEvent/1000) # convert spike times to ms

In [3]:
class event():
    '''
    This class provides a way to store, read, write and visualize spike event.
    Members:
        * ``x`` (numpy ``int`` array): `x` index of spike event.
        * ``y`` (numpy ``int`` array): `y` index of spike event
                                        (not used if the spatial dimension is 1).
        * ``p`` (numpy ``int`` array): `polarity` or `channel` index of spike event.
        * ``t`` (numpy ``double`` array): `timestamp` of spike event.
                                        Time is assumend to be in ms.
    '''
    def __init__(self, xEvent, yEvent, pEvent, tEvent):
        if yEvent is None:
            self.dim = 1
        else:
            self.dim = 2
        
        self.x = xEvent if type(xEvent) is np.ndarray else np.asarray(xEvent)
        # x spatial dimension
        self.y = yEvent if type(yEvent) is np.ndarray else np.asarray(yEvent)
        # y spatial dimension
        self.p = pEvent if type(pEvent) is np.ndarray else np.asarray(pEvent)
        # spike polarity
        self.t = tEvent if type(tEvent) is np.ndarray else np.asarray(tEvent)
        # time stamp in ms
        
        if not issubclass(self.x.dtype.type, np.integer):
            self.x = self.x.astype('int')
        if not issubclass(self.p.dtype.type, np.integer):
            self.p = self.p.astype('int')
        if self.dim == 2:
            if not issubclass(self.y.dtype.type, np.integer):
                self.y = self.y.astype('int')
        
        self.p -= self.p.min()
    
    def toSpikeTensor(self, emptyTensor, samplingTime=1, randomShift=False, binningMode='OR'):
        # Sampling time in ms
        '''
        Returns a numpy tensor that contains the spike events sampled in bins of `samplingTime`.
        The tensor is of dimension (channels, height, width, time) or``CHWT``.
        Arguments:
            * ``emptyTensor`` (``numpy or torch tensor``): an empty tensor to hold spike data 
            * ``samplingTime``: the width of time bin to use.
            * ``randomShift``: flag to shift the sample in time or not. Default: False.
            * ``binningMode``: the way spikes are binned. 'SUM' or 'OR' are supported. Default: 'OR'
        '''
        if randomShift is True:
            tSt = np.random.randint(
                max(
                    int(self.t.min() / samplingTime),
                    int(self.t.max() / samplingTime) - emptyTensor.shape[3],
                    emptyTensor.shape[3] - int(self.t.max() / samplingTime),
                    1,
                )
            )
        else:
            tSt = 0
        
        xEvent = np.round(self.x).astype(int)
        pEvent = np.round(self.p).astype(int)
        tEvent = np.round(self.t/samplingTime).astype(int) - tSt
        
        if self.dim == 1:
            validInd = np.argwhere( (xEvent < emptyTensor.shape[2]) &
                                    (pEvent < emptyTensor.shape[0]) &
                                    (tEvent < emptyTensor.shape[3]) &
                                    (xEvent >= 0) &
                                    (pEvent >= 0) &
                                    (tEvent >= 0))
            if binningMode.upper() == 'OR':
                emptyTensor[pEvent[validInd],
                            0, 
                            xEvent[validInd],
                            tEvent[validInd]] = 1/samplingTime
            elif binningMode.upper() == 'SUM':
                emptyTensor[pEvent[validInd],
                            0, 
                            xEvent[validInd],
                            tEvent[validInd]] += 1/samplingTime
            else:
                raise Exception('Unsupported binningMode. It was {}'.format(binningMode))

        elif self.dim == 2:
            yEvent = np.round(self.y).astype(int)
            validInd = np.argwhere((xEvent < emptyTensor.shape[2]) &
                                   (yEvent < emptyTensor.shape[1]) & 
                                   (pEvent < emptyTensor.shape[0]) &
                                   (tEvent < emptyTensor.shape[3]) &
                                   (xEvent >= 0) &
                                   (yEvent >= 0) & 
                                   (pEvent >= 0) &
                                   (tEvent >= 0))

            if binningMode.upper() == 'OR':
                emptyTensor[pEvent[validInd], 
                            yEvent[validInd],
                            xEvent[validInd],
                            tEvent[validInd]] = 1/samplingTime
            elif binningMode.upper() == 'SUM':
                emptyTensor[pEvent[validInd], 
                            yEvent[validInd],
                            xEvent[validInd],
                            tEvent[validInd]] += 1/samplingTime
            else:
                raise Exception('Unsupported binningMode. It was {}'.format(binningMode))
            
        return emptyTensor

In [18]:
def convert_to_npy(mode, out_dir):
    dataPath = '../../../DATA/N-MNIST/raw/NMNISTsmall/'
    if mode == 'train':
        samples = np.loadtxt(dataPath + 'train1K.txt').astype('int')
    elif mode == 'test':
        samples = np.loadtxt(dataPath + 'test100.txt').astype('int')
    else:
        print('typo!')
    for sample in samples:
        file_dir = sample[0]
        label  = sample[1]
        data = read2Dspikes(dataPath + str(file_dir) + '.bs2').toSpikeTensor(torch.zeros((2,34,34,300)),samplingTime=1.0)
        torch.save(data, out_dir + mode + '/' + str(file_dir) + '_' + str(label) + '.pt')
        print('The ' + str(file_dir) + ', label: ' + str(label) + ' successfully saved')

In [19]:
out_dir = '../../../DATA/N-MNIST/processed/NMNISTsmall/'

In [20]:
convert_to_npy('train', out_dir)

The 1, label: 5 successfully saved
The 2, label: 0 successfully saved
The 3, label: 4 successfully saved
The 4, label: 1 successfully saved
The 5, label: 9 successfully saved
The 6, label: 2 successfully saved
The 7, label: 1 successfully saved
The 8, label: 3 successfully saved
The 9, label: 1 successfully saved
The 10, label: 4 successfully saved
The 11, label: 3 successfully saved
The 12, label: 5 successfully saved
The 13, label: 3 successfully saved
The 14, label: 6 successfully saved
The 15, label: 1 successfully saved
The 16, label: 7 successfully saved
The 17, label: 2 successfully saved
The 18, label: 8 successfully saved
The 19, label: 6 successfully saved
The 20, label: 9 successfully saved
The 21, label: 4 successfully saved
The 22, label: 0 successfully saved
The 23, label: 9 successfully saved
The 24, label: 1 successfully saved
The 25, label: 1 successfully saved
The 26, label: 2 successfully saved
The 27, label: 4 successfully saved
The 28, label: 3 successfully saved
T

The 240, label: 5 successfully saved
The 241, label: 8 successfully saved
The 242, label: 6 successfully saved
The 243, label: 3 successfully saved
The 244, label: 7 successfully saved
The 245, label: 5 successfully saved
The 246, label: 8 successfully saved
The 247, label: 0 successfully saved
The 248, label: 9 successfully saved
The 249, label: 1 successfully saved
The 250, label: 0 successfully saved
The 251, label: 3 successfully saved
The 252, label: 1 successfully saved
The 253, label: 2 successfully saved
The 254, label: 2 successfully saved
The 255, label: 3 successfully saved
The 256, label: 3 successfully saved
The 257, label: 6 successfully saved
The 258, label: 4 successfully saved
The 259, label: 7 successfully saved
The 260, label: 5 successfully saved
The 261, label: 0 successfully saved
The 262, label: 6 successfully saved
The 263, label: 2 successfully saved
The 264, label: 7 successfully saved
The 265, label: 9 successfully saved
The 266, label: 8 successfully saved
T

The 468, label: 7 successfully saved
The 469, label: 7 successfully saved
The 470, label: 8 successfully saved
The 471, label: 1 successfully saved
The 472, label: 9 successfully saved
The 473, label: 2 successfully saved
The 474, label: 0 successfully saved
The 475, label: 5 successfully saved
The 476, label: 1 successfully saved
The 477, label: 2 successfully saved
The 478, label: 2 successfully saved
The 479, label: 7 successfully saved
The 480, label: 3 successfully saved
The 481, label: 5 successfully saved
The 482, label: 4 successfully saved
The 483, label: 9 successfully saved
The 484, label: 7 successfully saved
The 485, label: 1 successfully saved
The 486, label: 8 successfully saved
The 487, label: 3 successfully saved
The 488, label: 9 successfully saved
The 489, label: 6 successfully saved
The 490, label: 0 successfully saved
The 491, label: 3 successfully saved
The 492, label: 1 successfully saved
The 493, label: 1 successfully saved
The 494, label: 2 successfully saved
T

The 706, label: 8 successfully saved
The 707, label: 4 successfully saved
The 708, label: 9 successfully saved
The 709, label: 8 successfully saved
The 710, label: 0 successfully saved
The 711, label: 1 successfully saved
The 712, label: 1 successfully saved
The 713, label: 0 successfully saved
The 714, label: 2 successfully saved
The 715, label: 2 successfully saved
The 716, label: 3 successfully saved
The 717, label: 2 successfully saved
The 718, label: 4 successfully saved
The 719, label: 4 successfully saved
The 720, label: 5 successfully saved
The 721, label: 8 successfully saved
The 722, label: 6 successfully saved
The 723, label: 5 successfully saved
The 724, label: 7 successfully saved
The 725, label: 7 successfully saved
The 726, label: 8 successfully saved
The 727, label: 8 successfully saved
The 728, label: 9 successfully saved
The 729, label: 7 successfully saved
The 730, label: 4 successfully saved
The 731, label: 7 successfully saved
The 732, label: 3 successfully saved
T

The 947, label: 2 successfully saved
The 948, label: 5 successfully saved
The 949, label: 7 successfully saved
The 950, label: 0 successfully saved
The 951, label: 7 successfully saved
The 952, label: 1 successfully saved
The 953, label: 0 successfully saved
The 954, label: 3 successfully saved
The 955, label: 7 successfully saved
The 956, label: 6 successfully saved
The 957, label: 5 successfully saved
The 958, label: 0 successfully saved
The 959, label: 6 successfully saved
The 960, label: 1 successfully saved
The 961, label: 5 successfully saved
The 962, label: 1 successfully saved
The 963, label: 7 successfully saved
The 964, label: 8 successfully saved
The 965, label: 5 successfully saved
The 966, label: 0 successfully saved
The 967, label: 3 successfully saved
The 968, label: 4 successfully saved
The 969, label: 7 successfully saved
The 970, label: 7 successfully saved
The 971, label: 5 successfully saved
The 972, label: 7 successfully saved
The 973, label: 8 successfully saved
T

In [21]:
convert_to_npy('test', out_dir)

The 60001, label: 7 successfully saved
The 60002, label: 2 successfully saved
The 60003, label: 1 successfully saved
The 60004, label: 0 successfully saved
The 60005, label: 4 successfully saved
The 60006, label: 1 successfully saved
The 60007, label: 4 successfully saved
The 60008, label: 9 successfully saved
The 60009, label: 5 successfully saved
The 60010, label: 9 successfully saved
The 60011, label: 0 successfully saved
The 60012, label: 6 successfully saved
The 60013, label: 9 successfully saved
The 60014, label: 0 successfully saved
The 60015, label: 1 successfully saved
The 60016, label: 5 successfully saved
The 60017, label: 9 successfully saved
The 60018, label: 7 successfully saved
The 60019, label: 3 successfully saved
The 60020, label: 4 successfully saved
The 60021, label: 9 successfully saved
The 60022, label: 6 successfully saved
The 60023, label: 6 successfully saved
The 60024, label: 5 successfully saved
The 60025, label: 4 successfully saved
The 60026, label: 0 succe